In [6]:
from arcgis.gis import GIS
gis = GIS()

In [7]:
import pandas as pd
pd.set_option('display.max_rows', None)
from arcgis.features import SpatialDataFrame
import numpy as np

In [8]:
def table_to_data_frame(in_table, input_fields=None, where_clause=None):
    """Function will convert an arcgis table into a pandas dataframe with an object ID index, and the selected
    input fields using an arcpy.da.SearchCursor."""
    OIDFieldName = arcpy.Describe(in_table).OIDFieldName
    if input_fields:
        final_fields = [OIDFieldName] + input_fields
    else:
        final_fields = [field.name for field in arcpy.ListFields(in_table)]
    data = [row for row in arcpy.da.SearchCursor(in_table, final_fields, where_clause=where_clause)]
    fc_dataframe = pd.DataFrame(data, columns=final_fields)
    fc_dataframe = fc_dataframe.set_index(OIDFieldName, drop=True)
    return fc_dataframe

### SSSI Condition in Westminster Constituencies ###
I ran the below uisng the Pro tool first and then sent reult to python window to get the correct syntax to run in the notebook

In [1]:
arcpy.analysis.SummarizeWithin("Westminster_Constituencies_England", "Sites of Special Scientific Interest Condition Units (SSSI) England © Natural England", r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Condition_Constit", "KEEP_ALL", None, "ADD_SHAPE_SUM", "SQUAREKILOMETERS", "CONDITION", "NO_MIN_MAJ", "ADD_PERCENT", r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_CONDITION_Summary")

id,value
0,S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Condition_Constit
1,S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_CONDITION_Summary


In [9]:
constit_sssi_sum = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Condition_Constit"
sdf = SpatialDataFrame.from_featureclass(constit_sssi_sum)
sdf.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID,NAME,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1,SUM_Area_SQUAREKILOMETERS,Join_ID,SHAPE
0,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.000,VA,CIVIL VOTING AREA,,,8.970977,1,"{'rings': [[[483481.7999999998, 152923.6999999..."
1,2,Aldridge-Brownhills Boro Const,WMC,Westminster Constituency,ALDRIDGE-BROWNHILLS_BORO_CONST,1.0,2.0,119930.0,24629.0,E14000531,4401.652,0.000,VA,CIVIL VOTING AREA,,,1.003833,2,"{'rings': [[[404009.2000000002, 299236.4000000..."
2,3,Altrincham and Sale West Boro Const,WMC,Westminster Constituency,ALTRINCHAM_AND_SALE_WEST_BORO_CONST,1.0,2.0,70266.0,24649.0,E14000532,5092.943,0.000,VA,CIVIL VOTING AREA,,,0.816205,3,"{'rings': [[[368282, 388925.80000000075], [368..."
3,4,Amber Valley Co Const,WMC,Westminster Constituency,AMBER_VALLEY_CO_CONST,1.0,2.0,70564.0,24766.0,E14000533,12464.642,0.000,VA,CIVIL VOTING AREA,,,0.201765,4,"{'rings': [[[439005.7000000002, 342789], [4388..."
4,5,Arundel and South Downs Co Const,WMC,Westminster Constituency,ARUNDEL_AND_SOUTH_DOWNS_CO_CONST,1.0,2.0,124062.0,25172.0,E14000534,64651.292,126.204,VA,CIVIL VOTING AREA,,,26.813789,5,"{'rings': [[[491410.7000000002, 120536.3000000..."


In [10]:
sssi_con = r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_CONDITION_Summary"
df = table_to_data_frame(sssi_con)
df.head()

,Join_ID,CONDITION,SUM_Area_SQUAREKILOMETERS,PercentArea
OBJECTID,,,,
1,1,FAVOURABLE,0.823493,1.553439
2,1,UNFAVOURABLE DECLINING,0.114294,0.215605
3,1,UNFAVOURABLE NO CHANGE,0.000003,0.000005
4,1,UNFAVOURABLE RECOVERING,8.033187,15.153833
5,2,FAVOURABLE,0.090011,0.204330


In [11]:
constit_condition = pd.merge(df,sdf,on='Join_ID')
constit_condition.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,Join_ID,CONDITION,SUM_Area_SQUAREKILOMETERS_x,PercentArea,OBJECTID,NAME,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1,SUM_Area_SQUAREKILOMETERS_y,SHAPE
0,1,FAVOURABLE,0.823493,1.553439,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
1,1,UNFAVOURABLE DECLINING,0.114294,0.215605,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
2,1,UNFAVOURABLE NO CHANGE,0.000003,0.000005,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
3,1,UNFAVOURABLE RECOVERING,8.033187,15.153833,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
4,2,FAVOURABLE,0.090011,0.204330,2,Aldridge-Brownhills Boro Const,WMC,Westminster Constituency,ALDRIDGE-BROWNHILLS_BORO_CONST,1.0,2.0,119930.0,24629.0,E14000531,4401.652,0.0,VA,CIVIL VOTING AREA,,,1.003833,"{'rings': [[[404009.2000000002, 299236.4000000..."


In [12]:
constit_condition['% Condition'] = (constit_condition['SUM_Area_SQUAREKILOMETERS_x'] / constit_condition['SUM_Area_SQUAREKILOMETERS_y']*100)
constit_condition['Area Ha'] = (constit_condition['SUM_Area_SQUAREKILOMETERS_x'] *100)
constit_condition[['NAME', 'CONDITION','SUM_Area_SQUAREKILOMETERS_x','SUM_Area_SQUAREKILOMETERS_y','Area Ha','% Condition']]

,NAME,CONDITION,SUM_Area_SQUAREKILOMETERS_x,SUM_Area_SQUAREKILOMETERS_y,Area Ha,% Condition
0,Aldershot Boro Const,FAVOURABLE,0.823493,8.970977,82.349264,9.179520
1,Aldershot Boro Const,UNFAVOURABLE DECLINING,0.114294,8.970977,11.429447,1.274047
2,Aldershot Boro Const,UNFAVOURABLE NO CHANGE,0.000003,8.970977,0.000287,0.000032
3,Aldershot Boro Const,UNFAVOURABLE RECOVERING,8.033187,8.970977,803.318713,89.546401
4,Aldridge-Brownhills Boro Const,FAVOURABLE,0.090011,1.003833,9.001066,8.966695
5,Aldridge-Brownhills Boro Const,UNFAVOURABLE DECLINING,0.292379,1.003833,29.237887,29.126240
6,Aldridge-Brownhills Boro Const,UNFAVOURABLE NO CHANGE,0.061210,1.003833,6.120957,6.097584
7,Aldridge-Brownhills Boro Const,UNFAVOURABLE RECOVERING,0.560234,1.003833,56.023409,55.809480
8,Altrincham and Sale West Boro Const,FAVOURABLE,0.816205,0.816205,81.620495,100.000000
9,Amber Valley Co Const,DESTROYED,0.006011,0.201765,0.601149,2.979453


Need to create table of area of SSSI for each constituency as tables further below show area for favourable and unfavourable not overall which is required for dashboard list.

In [14]:
constit_all = constit_condition.groupby("NAME", as_index=False)["Area Ha"].sum()
constit_all

,NAME,Area Ha
0,Aldershot Boro Const,897.097711
1,Aldridge-Brownhills Boro Const,100.383320
2,Altrincham and Sale West Boro Const,81.620495
3,Amber Valley Co Const,20.176489
4,Arundel and South Downs Co Const,2681.378900
5,Ashfield Co Const,105.259625
6,Ashford Co Const,1282.650435
7,Ashton-under-Lyne Boro Const,6.811922
8,Aylesbury Co Const,406.575955
9,Banbury Co Const,199.204397


In [15]:
#convert to gdb table
x = np.array(np.rec.fromrecords(constit_all.values))
names = constit_all.dtypes.index.tolist()
x.dtype.names = tuple(names)
arcpy.da.NumPyArrayToTable(x, r'S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\constit_sssi_all')

In [8]:
constit_fav = constit_condition.query('CONDITION == "FAVOURABLE"')
constit_fav = constit_fav.rename(columns = {'% Condition': '% Favourable'}, inplace = False)
constit_fav_final = constit_fav[['NAME','Area Ha','% Favourable']]
constit_fav_final

,NAME,Area Ha,% Favourable
0,Aldershot Boro Const,82.349264,9.179520
4,Aldridge-Brownhills Boro Const,9.001066,8.966695
8,Altrincham and Sale West Boro Const,81.620495,100.000000
10,Amber Valley Co Const,17.608767,87.273692
14,Arundel and South Downs Co Const,1663.883512,62.053278
18,Ashfield Co Const,20.682966,19.649477
22,Ashford Co Const,806.950304,62.912722
29,Aylesbury Co Const,274.807452,67.590680
34,Banbury Co Const,143.863295,72.218935
37,Barnsley East Co Const,15.841655,100.000000


In [9]:
constit_unfav = constit_condition.query('CONDITION.str.contains("UNFAVOURABLE")', engine='python')
constit_unfav = constit_unfav.rename(columns = {'% Condition': '% Unfavourable'}, inplace = False)
constit_unfav[['NAME', 'CONDITION','Area Ha','% Unfavourable']]

,NAME,CONDITION,Area Ha,% Unfavourable
1,Aldershot Boro Const,UNFAVOURABLE DECLINING,11.429447,1.274047
2,Aldershot Boro Const,UNFAVOURABLE NO CHANGE,0.000287,0.000032
3,Aldershot Boro Const,UNFAVOURABLE RECOVERING,803.318713,89.546401
5,Aldridge-Brownhills Boro Const,UNFAVOURABLE DECLINING,29.237887,29.126240
6,Aldridge-Brownhills Boro Const,UNFAVOURABLE NO CHANGE,6.120957,6.097584
7,Aldridge-Brownhills Boro Const,UNFAVOURABLE RECOVERING,56.023409,55.809480
11,Amber Valley Co Const,UNFAVOURABLE DECLINING,1.109126,5.497123
12,Amber Valley Co Const,UNFAVOURABLE RECOVERING,0.857447,4.249732
15,Arundel and South Downs Co Const,UNFAVOURABLE DECLINING,38.500918,1.435863
16,Arundel and South Downs Co Const,UNFAVOURABLE NO CHANGE,5.197432,0.193834


In [10]:
constit_unfav_final = constit_unfav.groupby("NAME", as_index=False)["Area Ha","% Unfavourable"].sum()
constit_unfav_final

__main__:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


,NAME,Area Ha,% Unfavourable
0,Aldershot Boro Const,814.748447,90.820480
1,Aldridge-Brownhills Boro Const,91.382254,91.033305
2,Amber Valley Co Const,1.966573,9.746855
3,Arundel and South Downs Co Const,1016.972173,37.927209
4,Ashfield Co Const,84.576659,80.350523
5,Ashford Co Const,475.700132,37.087278
6,Ashton-under-Lyne Boro Const,6.811922,100.000000
7,Aylesbury Co Const,131.768503,32.409320
8,Banbury Co Const,46.732691,23.459669
9,Barrow and Furness Co Const,2260.044063,15.944803


dataframe needs to be converted to numpy array before being converted to GDB table (may be better ways to do this)

In [27]:
x = np.array(np.rec.fromrecords(constit_unfav_final.values))
names = constit_unfav_final.dtypes.index.tolist()
x.dtype.names = tuple(names)
arcpy.da.NumPyArrayToTable(x, r'S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\constit_sssi_unfavoruable')

### SSSI Assessed in last 6 years in constituencies ###

prior to the step below I created a new field for SSSI condition data to flag if assessed in last 6 years. as adding a new field I had to make a copy of the Living Atlas layer

In [28]:
arcpy.analysis.SummarizeWithin("Westminster_Constituencies_England", "England_SSSI_Condition_Sep2020", r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Assessed_Constit", "KEEP_ALL", None, "ADD_SHAPE_SUM", "SQUAREKILOMETERS", "Assessed_last_6years", "NO_MIN_MAJ", "ADD_PERCENT", r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_ASSESSED_Summary")


id,value
0,S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Assessed_Constit
1,S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_ASSESSED_Summary


In [30]:
constit_sssi_ass_sum = "S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_SSSI_Assessed_Constit"
asdf = SpatialDataFrame.from_featureclass(constit_sssi_ass_sum)
asdf.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,OBJECTID,NAME,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1,SUM_Area_SQUAREKILOMETERS,Join_ID,SHAPE
0,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.000,VA,CIVIL VOTING AREA,,,8.970977,1,"{'rings': [[[483481.7999999998, 152923.6999999..."
1,2,Aldridge-Brownhills Boro Const,WMC,Westminster Constituency,ALDRIDGE-BROWNHILLS_BORO_CONST,1.0,2.0,119930.0,24629.0,E14000531,4401.652,0.000,VA,CIVIL VOTING AREA,,,1.003833,2,"{'rings': [[[404009.2000000002, 299236.4000000..."
2,3,Altrincham and Sale West Boro Const,WMC,Westminster Constituency,ALTRINCHAM_AND_SALE_WEST_BORO_CONST,1.0,2.0,70266.0,24649.0,E14000532,5092.943,0.000,VA,CIVIL VOTING AREA,,,0.816205,3,"{'rings': [[[368282, 388925.80000000075], [368..."
3,4,Amber Valley Co Const,WMC,Westminster Constituency,AMBER_VALLEY_CO_CONST,1.0,2.0,70564.0,24766.0,E14000533,12464.642,0.000,VA,CIVIL VOTING AREA,,,0.201765,4,"{'rings': [[[439005.7000000002, 342789], [4388..."
4,5,Arundel and South Downs Co Const,WMC,Westminster Constituency,ARUNDEL_AND_SOUTH_DOWNS_CO_CONST,1.0,2.0,124062.0,25172.0,E14000534,64651.292,126.204,VA,CIVIL VOTING AREA,,,26.813789,5,"{'rings': [[[491410.7000000002, 120536.3000000..."


In [31]:
sssi_ass = r"S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\Eng_Constit_SSSI_ASSESSED_Summary"
adf = table_to_data_frame(sssi_ass)
adf.head()

,Join_ID,Assessed_last_6years,SUM_Area_SQUAREKILOMETERS,PercentArea
OBJECTID,,,,
1,1,No,8.431304,15.904843
2,1,Yes,0.539673,1.018041
3,2,No,0.573231,1.301271
4,2,Yes,0.430602,0.977494
5,3,No,0.816205,1.601367


In [33]:
constit_assessed = pd.merge(adf,asdf,on='Join_ID')
constit_assessed.head()

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\arcgis\features\_data\geodataset\geodataframe.py:221: UserWarning: SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.
  warnings.warn("SpatialDataFrame has been deprecated.  Please switch to the GeoAccessor/GeoSeriesAccessor.")


,Join_ID,Assessed_last_6years,SUM_Area_SQUAREKILOMETERS_x,PercentArea,OBJECTID,NAME,AREA_CODE,DESCRIPTIO,FILE_NAME,NUMBER,NUMBER0,POLYGON_ID,UNIT_ID,CODE,HECTARES,AREA,TYPE_CODE,DESCRIPT0,TYPE_COD0,DESCRIPT1,SUM_Area_SQUAREKILOMETERS_y,SHAPE
0,1,No,8.431304,15.904843,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
1,1,Yes,0.539673,1.018041,1,Aldershot Boro Const,WMC,Westminster Constituency,ALDERSHOT_BORO_CONST,1.0,2.0,70075.0,25022.0,E14000530,5297.816,0.0,VA,CIVIL VOTING AREA,,,8.970977,"{'rings': [[[483481.7999999998, 152923.6999999..."
2,2,No,0.573231,1.301271,2,Aldridge-Brownhills Boro Const,WMC,Westminster Constituency,ALDRIDGE-BROWNHILLS_BORO_CONST,1.0,2.0,119930.0,24629.0,E14000531,4401.652,0.0,VA,CIVIL VOTING AREA,,,1.003833,"{'rings': [[[404009.2000000002, 299236.4000000..."
3,2,Yes,0.430602,0.977494,2,Aldridge-Brownhills Boro Const,WMC,Westminster Constituency,ALDRIDGE-BROWNHILLS_BORO_CONST,1.0,2.0,119930.0,24629.0,E14000531,4401.652,0.0,VA,CIVIL VOTING AREA,,,1.003833,"{'rings': [[[404009.2000000002, 299236.4000000..."
4,3,No,0.816205,1.601367,3,Altrincham and Sale West Boro Const,WMC,Westminster Constituency,ALTRINCHAM_AND_SALE_WEST_BORO_CONST,1.0,2.0,70266.0,24649.0,E14000532,5092.943,0.0,VA,CIVIL VOTING AREA,,,0.816205,"{'rings': [[[368282, 388925.80000000075], [368..."


In [35]:
constit_assessed['% assessed'] = (constit_assessed['SUM_Area_SQUAREKILOMETERS_x'] / constit_assessed['SUM_Area_SQUAREKILOMETERS_y']*100)
constit_assessed['Area Ha'] = (constit_assessed['SUM_Area_SQUAREKILOMETERS_x'] *100)
constit_assessed[['NAME', 'Assessed_last_6years','SUM_Area_SQUAREKILOMETERS_x','SUM_Area_SQUAREKILOMETERS_y','Area Ha','% assessed']]

,NAME,Assessed_last_6years,SUM_Area_SQUAREKILOMETERS_x,SUM_Area_SQUAREKILOMETERS_y,Area Ha,% assessed
0,Aldershot Boro Const,No,8.431304,8.970977,843.130436,93.984237
1,Aldershot Boro Const,Yes,0.539673,8.970977,53.967275,6.015763
2,Aldridge-Brownhills Boro Const,No,0.573231,1.003833,57.323103,57.104211
3,Aldridge-Brownhills Boro Const,Yes,0.430602,1.003833,43.060217,42.895789
4,Altrincham and Sale West Boro Const,No,0.816205,0.816205,81.620495,100.000000
5,Amber Valley Co Const,No,0.201765,0.201765,20.176489,100.000000
6,Arundel and South Downs Co Const,No,25.168837,26.813789,2516.883729,93.865277
7,Arundel and South Downs Co Const,Yes,1.644952,26.813789,164.495172,6.134723
8,Ashfield Co Const,No,0.915484,1.052596,91.548367,86.973868
9,Ashfield Co Const,Yes,0.137113,1.052596,13.711257,13.026132


In [37]:
constit_not_assessed = constit_assessed.query('Assessed_last_6years == "No"')
constit_not_assessed = constit_not_assessed.rename(columns = {'% assessed': '% not assessed'}, inplace = False)
constit_not_assessed_final = constit_not_assessed[['NAME','Area Ha','% not assessed']]
constit_not_assessed_final

,NAME,Area Ha,% not assessed
0,Aldershot Boro Const,843.130436,93.984237
2,Aldridge-Brownhills Boro Const,57.323103,57.104211
4,Altrincham and Sale West Boro Const,81.620495,100.000000
5,Amber Valley Co Const,20.176489,100.000000
6,Arundel and South Downs Co Const,2516.883729,93.865277
8,Ashfield Co Const,91.548367,86.973868
10,Ashford Co Const,861.170285,67.139905
12,Ashton-under-Lyne Boro Const,6.811922,100.000000
13,Aylesbury Co Const,388.089550,95.453149
15,Banbury Co Const,190.434538,95.597558


In [40]:
y = np.array(np.rec.fromrecords(constit_not_assessed_final.values))
names = constit_not_assessed_final.dtypes.index.tolist()
y.dtype.names = tuple(names)
arcpy.da.NumPyArrayToTable(y, r'S:\DataUnit\DptShare\Consultancy\Lodge\Conservation and Policy\DavidHampson\Lost_Decade_PA_analysis\ArcGIS Pro\Protected_Area_Analysis_UK\Protected_Area_Analysis_UK.gdb\constit_sssi_not_assesed_last_6years')